# 01 Load and Explore Collection

We have a lot of different events generated for different detectors. This notebook shows you with examples the functionalities of the event classes. Particularly, how to load events generated by olympus and how to modify the events to be prepared for the dataset generation at a later point

## Table of Contents
* [Import Dependencies](#dependencies)
* [Load Event Collection from Olympus](#load-events)
* [Explore the Event Class](#event)
* [Explore the EventCollection Class](#event-collection)

## Import Dependencies <a class="anchor" id="dependencies"></a>

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
sys.path.append("../../olympus")
from plotly.subplots import make_subplots

import apollo

from apollo.data.importers import EventCollectionImporter
from apollo.utils.detector_helpers import get_line_detector
from apollo.data.configs import Interval, HistogramConfig
from apollo.data.events import EventTimeframeMode
from apollo.visualization.events import plot_histogram

ImportError: cannot import name 'make_line' from 'olympus.event_generation.detector' (/home/janik/projects/master/olympus/olympus/event_generation/detector.py)

## Load Event Collection from Olympus <a class="anchor" id="load-events"></a>

Before we can load the events we have to initialize the detector. The `detector_helpers` are quite useful, as they instanciate the detector like Olympus would.

In [2]:
detector = get_line_detector()

NameError: name 'get_line_detector' is not defined

Now we can continue and load the events from the pickle files or folders.

In [ ]:
event_collection = EventCollectionImporter.from_pickle(
    "../../data/all/events_track_0.pickle", detector=detector
)
# When you want to load the whole folder, use:
# event_collection = EventCollectionImporter.from_folder('../../data/all', detector=detector)

## Explore the Event Class<a class="anchor" id="event"></a>

The event class is the backbone of the individual event. Its functionalities mostly propagate to the event collection. Let's first get a cool event to play width.

In [ ]:
current_event = None

for event in event_collection.events:
    if event.number_of_hits > 100:
        current_event = event
        break

current_event.as_features()

For creating random datasets with an event at multiple plases, it is useful to be able to move the event around. 

### Explore the TimeFrameModes

As there are multiple reference points for moving the event around, it makes sense to be able to reference multiple of them:
* `START_TIME`: The events start time is the reference
* `CONTAINS_HIT`: Makes sure that at least one hit is included
* `CONTAINS_EVENT`: Makes sure, if possible, that the whole event is included
* `CONTAINS_PERCENTILE`: Makes sure that a given percentile of hits are included (extra parameter percentile necessary)

### Redistribute the Event and Create histograms

Let's get it moving and create histograms according to the events.

In [ ]:
interval = Interval(0, 1000)
histogram_config = HistogramConfig(start=interval.start, end=interval.end, bin_size=10)

raw_histogram = current_event.get_histogram(histogram_config)

current_event.redistribute(interval, is_in_timeframe_mode=EventTimeframeMode.START_TIME)
print("By Start Time: ", current_event.time)

start_time_histogram = current_event.get_histogram(histogram_config)

current_event.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_HIT
)
print("By Contains Hit: ", current_event.time)

contains_hit_histogram = current_event.get_histogram(histogram_config)

current_event.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_EVENT
)
print("By Contains Event: ", current_event.time)

contains_event_histogram = current_event.get_histogram(histogram_config)

current_event.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_PERCENTAGE, percentile=50
)
print("By Contains Percentile: ", current_event.time)

contains_percentile_histogram = current_event.get_histogram(histogram_config)

Let's have a look at it. How do the different events compare?

In [ ]:
fig = make_subplots(
    3,
    2,
    subplot_titles=(
        "Raw Event",
        "",
        "START TIME Event",
        "CONTAINS HIT Event",
        "CONTAINS EVENT Event",
        "CONTAINS PERCENTAGE Event",
    ),
)


def plot_histogram_helper(histogram, row, column):
    fig.add_trace(plot_histogram(histogram), row=row, col=column)


plot_histogram_helper(raw_histogram, 1, 1)

plot_histogram_helper(start_time_histogram, 2, 1)

plot_histogram_helper(contains_hit_histogram, 2, 2)

plot_histogram_helper(contains_event_histogram, 3, 1)

plot_histogram_helper(contains_percentile_histogram, 3, 2)

fig.update_traces(showlegend=False, showscale=False)

fig.show()

## Explore the EventCollection <a class="anchor" id="event-collection"></a>

### Access the Event Collection


In [ ]:
# Subset with array like indexing
event_collection2 = event_collection[1:3]
print("Nr. of events: ", len(event_collection), ", ", type(event_collection))
print("Nr. of events: ", len(event_collection2), ", ", type(event_collection2))

In [ ]:
# Concatenation like an array
event_collection3 = event_collection + event_collection2
print("Nr. of events: ", len(event_collection3), ", ", type(event_collection3))

### Redistribute and show

Like the event class, the collection can be redistributed and showed

In [ ]:
interval = Interval(0, 1000)
histogram_config = HistogramConfig(start=interval.start, end=interval.end, bin_size=10)

raw_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.START_TIME
)

start_time_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_HIT
)

contains_hit_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_EVENT
)

contains_event_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_PERCENTAGE, percentile=50
)

contains_percentile_histogram_collection = event_collection.get_histogram(
    histogram_config
)

Let's have a look again

In [ ]:
fig = make_subplots(
    3,
    2,
    subplot_titles=(
        "Raw Event",
        "",
        "START TIME Event",
        "CONTAINS HIT Event",
        "CONTAINS EVENT Event",
        "CONTAINS PERCENTAGE Event",
    ),
)


def plot_histogram_helper(histogram, row, column):
    fig.add_trace(plot_histogram(histogram), row=row, col=column)


plot_histogram_helper(raw_histogram_collection, 1, 1)

plot_histogram_helper(start_time_histogram_collection, 2, 1)

plot_histogram_helper(contains_hit_histogram_collection, 2, 2)

plot_histogram_helper(contains_event_histogram_collection, 3, 1)

plot_histogram_helper(contains_percentile_histogram_collection, 3, 2)

fig.update_traces(showlegend=False, showscale=False)

fig.show()

In [ ]:
# Subset with array like indexing
event_collection2 = event_collection[1:3]
print("Nr. of events: ", len(event_collection), ", ", type(event_collection))
print("Nr. of events: ", len(event_collection2), ", ", type(event_collection2))

In [ ]:
# Concatenation like an array
event_collection3 = event_collection + event_collection2
print("Nr. of events: ", len(event_collection3), ", ", type(event_collection3))

### Redistribute and show

Like the event class, the collection can be redistributed and showed

In [ ]:
interval = Interval(0, 1000)
histogram_config = HistogramConfig(start=interval.start, end=interval.end, bin_size=10)

raw_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.START_TIME
)

start_time_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_HIT
)

contains_hit_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_EVENT
)

contains_event_histogram_collection = event_collection.get_histogram(histogram_config)

event_collection.redistribute(
    interval, is_in_timeframe_mode=EventTimeframeMode.CONTAINS_PERCENTAGE, percentile=50
)

contains_percentile_histogram_collection = event_collection.get_histogram(
    histogram_config
)

Let's have a look again

In [ ]:
fig = make_subplots(
    3,
    2,
    subplot_titles=(
        "Raw Event",
        "",
        "START TIME Event",
        "CONTAINS HIT Event",
        "CONTAINS EVENT Event",
        "CONTAINS PERCENTAGE Event",
    ),
)


def plot_histogram_helper(histogram, row, column):
    fig.add_trace(plot_histogram(histogram), row=row, col=column)


plot_histogram_helper(raw_histogram_collection, 1, 1)

plot_histogram_helper(start_time_histogram_collection, 2, 1)

plot_histogram_helper(contains_hit_histogram_collection, 2, 2)

plot_histogram_helper(contains_event_histogram_collection, 3, 1)

plot_histogram_helper(contains_percentile_histogram_collection, 3, 2)

fig.update_traces(showlegend=False, showscale=False)

fig.show()